In [1]:
# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import boxcox
from statsmodels.tsa.statespace.tools import diff
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

This file is a demo of how it could look in a real situation with forecast weather data (may require adjustments!)

In [ ]:
# the reading of the data depends on on how it is retrieved
# it is either all in one csv file or you have 2 files,
# one for the historic data that is used for the training
# and one for the forecast weather data that is used for forecasting energy
# the data should be cleaned first

# provide proper csv file names
df_historic = pd.read_csv('(...).csv')
df_historic['date'] = pd.to_datetime(df_historic['(name of date column)'])
forecast_weather = pd.read_csv('(...).csv')
forecast_weather = pd.to_datetime(forecast_weather['(name of date column)'])

ACF and PACF plots used to determine p, q values

In [ ]:
# how to look for orders of sarimax:
# look at lower order lags < 4 for p and q values
# p value is significant lag in lower order of PACF plot, q value is significant lag in lower order ACF plot
# seasonal P value is significant lag of seasonal lags (multiples of 4) in PACF plot, seasonal Q value is significant lag of seasonal lags in ACF plot
def plot_acf_pacf(df: pd.DataFrame) -> None:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), facecolor='w')
    
    # Plot ACF
    plot_acf(df, ax=ax1, zero=False)
    ax1.set_title("ACF plot for Seasonal Differenced Series Energy Consumption", fontsize=20)
    ax1.set_ylabel("ACF", fontsize=18)
    ax1.set_xlabel("Lag", fontsize=18)
    ax1.tick_params(axis='both', labelsize=14)
    ax1.grid(alpha=1)

    # Plot PACF
    plot_pacf(df, ax=ax2, zero=False)
    ax2.set_title("PACF plot for Seasonal Differenced Energy Consumption", fontsize=20) 
    ax2.set_ylabel("PACF", fontsize=18)
    ax2.set_xlabel("Lag", fontsize=18)
    ax2.tick_params(axis='both', labelsize=14)
    ax2.grid(alpha=1)

    plt.tight_layout()

    plt.show()

seasonal_diff = diff(df_historic['(energy consumption column name)'], k_diff=0, k_seasonal_diff=1, seasonal_periods=4)
plot_acf_pacf(seasonal_diff)

Data preparation

In [ ]:
missing_date_start = '2023-05-13'
missing_date_end = '2023-10-15 06:00'

In [ ]:
# this data contains a missing section between dates '2023-05-14' and '2023-10-15 06:00'
# you might need to change this and/or add other missing parts
_ , lambda_value = boxcox(df_historic.loc[(df_historic['date'] <= missing_date_start) | (df_historic['date'] >= missing_date_end), 'energy_consumption'] + 0.001)
print(lambda_value)

# -1. is a reciprocal
# -.5 is a recriprocal square root
# 0.0 is a log transformation
# .5 is a square toot transform and
# 1.0 is no transform.

# custom root function lambda value
_BASE_ = 1.05
def custom_root(x, base):
    return x ** (1 / base)
def reverse_custom_root(y, base):
    return y ** base

In [ ]:
missing_date_start = '2023-05-13'
missing_date_end = '2023-10-15 06:00'

In [ ]:
# adjust data
# this data contains a missing section between dates '2023-05-14' and '2023-10-15 06:00'
# you might need to change this and/or add other missing parts
df_historic.loc[(df_historic['date'] > missing_date_start) & (df_historic['date'] < missing_date_end), '(energy consumption column name)'] = np.nan
df_historic['(energy consumption column name)'] = df_historic['(energy consumption column name)'].apply(lambda x: x - 3 if x > 12 else x)
df_historic['(energy consumption column name)'] = custom_root(df_historic['(energy consumption column name)'], _BASE_)

# add holiday and workday effects
holiday_start = "2023-12-22"
holiday_end = "2023-12-29"
df_historic['effects'] = np.where((df_historic['date'].dt.weekday < 5) & ~((df_historic['date'] >= holiday_start) & (df_historic['date'] <= holiday_end)),1,0)

df_historic.set_index('date', inplace=True)
forecast_weather.set_index('date', inplace=True)
# instead of creating a train test split, now the historic data will be used to train the model
# and the weather forecast will be used to make energy forecasts


Gridsearch

In [ ]:
# explanatory variables, depends on the column names and used variables
# make sure the column of the historic weather are the same as the forecast weather data
exogs = ['temperature','humidity','evaporation','wind_speed','effects']

In [ ]:
# caution, running the gridsearch may take a while, recommend running this only once for search and then commening out

orders = [(p, 0, q) for p in range(0, 3) for q in range(0, 3)]
seasonals = [(P, 1, Q, 4) for P in range(1, 3) for Q in range(2, 3)]
result_GS2 = []
for order in orders:
    for seasonal in seasonals:

        model = SARIMAX(df_historic['(energy consumption column name)'], exog=df_historic[exogs], order=order, seasonal_order=seasonal)
        results = model.fit(disp=False, method='powell')
        
        result_GS2.append({
            'order': order,
            'seasonal_order': seasonal,
            'AIC': results.aic,
        })

results_df = pd.DataFrame(result_GS2)

In [ ]:
print(results_df.sort_values(by='AIC', ascending=True))

Model SARIMAX

In [ ]:
# best orders from grid search with lowest AIC: 
# train model
model = SARIMAX(df_historic['(energy consumption column name)'], order=(2,0,0), seasonal_order=(2,1,2,4), exog=df_historic[exogs])
model_fit = model.fit(disp=False, method='powell')
print(model_fit.summary())

In [ ]:
# get forecast and values
forecast = model_fit.get_forecast(steps=len(forecast_weather), exog=forecast_weather[exogs])
forecast_values = reverse_custom_root(forecast.predicted_mean.clip(lower=0),_BASE_)
forecast_ci = reverse_custom_root(forecast.conf_int().clip(lower=0),_BASE_)

In [ ]:
# plot with previous data could look something like this

train_subset = df_historic[-int(len(df_historic) * 0.2):]

plt.figure(figsize=(10, 2.5))
plt.plot(train_subset.index, reverse_custom_root(train_subset['(energy consumption column name)'], _BASE_), label='Training', color='black')
plt.plot(forecast_weather.index, forecast_values, label='Forecast', color='red')
plt.fill_between(forecast_weather.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], color='pink', alpha=0.4)
plt.legend()
plt.title("Forecasted Heat Pump Energy Consumption (with Training Data)", fontsize=16)
plt.xlabel("Date", fontsize=14) 
plt.ylabel("kWh", fontsize=14)  

plt.tight_layout()
plt.show()

In [ ]:
# evaluation metrics cant be performed without test data